In [1]:
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dataset_path = "/content/drive/MyDrive/probing"

### Dataset

In [ ]:
from torch.utils.data import Dataset

class ActivationsDataset(Dataset):
    def __init__(self, data_path, layer: int, batches_to_load: int):
        self.data_path = data_path
        self.layer = layer
        self.batches_to_load = batches_to_load

        self.activations = activations
        self.labels = labels

    def _load_activations(self):
        activations = torch.load(
            f"{self.data_path}/layer_activations_batch_{i}.pt" 
            for i in range(self.batches_to_load)
        )
        activations = torch.cat(
            [torch.load(f"{self.data_path}/layer_{self.layer}_activations_batch_{i}.pt") 
             for i in range(self.batches_to_load)], dim=0
        )
        return activations

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.activations[idx], self.labels[idx]

In [ ]:
for i in range(64):
    activations_path = f"/content/drive/MyDrive/probing/activations-0/layers_activations_batch_{i}.pt"

    batch = torch.load(activations_path)
    examples = []
    for activations in batch:
        for layer in range(len(activations)):
            dataset_example = {
                "activations": activations[layer]
            }

### Probe model

In [6]:
from torch import nn

class Probe(nn.Module):
    def __init__(self, input_dim, output_dim=1):
        super().__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.linear(x)

### Training Loop

In [18]:
x = torch.randn(3, 50)
y = torch.tensor([[1.0], [0.0], [1.0]])

learning_rate = 0.001
batch_size = 1
epochs = 100

model = Probe(input_dim=50)
loss = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    for i in range(0, len(x), batch_size):
        batch = x[i:i+batch_size]
        # inputs = batch[:, :-1]
        targets = y[i:i+batch_size]

        optimizer.zero_grad()
        outputs = model(batch)
        l = loss(outputs, targets)
        l.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {l.item()}")

Epoch 1/100, Loss: 0.9886660575866699
Epoch 2/100, Loss: 0.9498652815818787
Epoch 3/100, Loss: 0.9104343056678772
Epoch 4/100, Loss: 0.8715340495109558
Epoch 5/100, Loss: 0.8335779905319214
Epoch 6/100, Loss: 0.7967727184295654
Epoch 7/100, Loss: 0.7612372636795044
Epoch 8/100, Loss: 0.727043867111206
Epoch 9/100, Loss: 0.6942340731620789
Epoch 10/100, Loss: 0.6628273725509644
Epoch 11/100, Loss: 0.6328264474868774
Epoch 12/100, Loss: 0.604220449924469
Epoch 13/100, Loss: 0.5769875645637512
Epoch 14/100, Loss: 0.5510972142219543
Epoch 15/100, Loss: 0.5265117287635803
Epoch 16/100, Loss: 0.50318843126297
Epoch 17/100, Loss: 0.48108041286468506
Epoch 18/100, Loss: 0.4601382613182068
Epoch 19/100, Loss: 0.44031065702438354
Epoch 20/100, Loss: 0.4215453565120697
Epoch 21/100, Loss: 0.40379029512405396
Epoch 22/100, Loss: 0.38699349761009216
Epoch 23/100, Loss: 0.37110430002212524
Epoch 24/100, Loss: 0.3560730218887329
Epoch 25/100, Loss: 0.34185168147087097
Epoch 26/100, Loss: 0.3283942043